In [9]:
# Useful starting lines
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.linear_model as sk
import re
from glove_routines import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [197]:

word = "lol wutwut (8 lol ? <user> lol , quoi ahah lol wutwut lol lol"
smiley = ":')"
print(re.escape(smiley))
test = word.split()
listesc = ".^$*+?{}[]\|()" #list of all escaped charactes
if('\(8' in listesc):
    print("loel")
pd = "\?" 
if re.search(r"(?:(?:(?:\b)|^)"+pd+"(?:(?=\b)|$)|(?:^|(?:\s))"+pd+"(?:$|(?=\s)))",word):
    count = re.findall(r"(?:(?:(?:\b)|^)"+pd+"(?:(?=\b)|$)|(?:^|(?:\s))"+pd+"(?:$|(?=\s)))",word)
    print(count)

\:\'\)
[' ?']


In [200]:
    pos_train = open('data/pos_train.txt').readlines()
    neg_train = open('data/neg_train.txt').readlines()
    embeddings = np.load('data/embeddings.npy')

    
    #count number of word/tweet and store it for both positive set and negative set
    word_nbr_per_tweet_pos = np.zeros(np.shape(pos_train)[0])
    for j in range(0,np.shape(pos_train)[0]):
        tweet = pos_train[j]
        size = len(tweet.split())
        word_nbr_per_tweet_pos[j] = size
    
    word_nbr_per_tweet_neg = np.zeros(np.shape(neg_train)[0])
    for j in range(0,np.shape(neg_train)[0]):
        tweet = neg_train[j]
        size = len(tweet.split())
        word_nbr_per_tweet_neg[j] = size
    
    print("counting ended")
    
    i=0
    pos_mask = np.zeros(np.shape(embeddings)[1]+1)
    pos_mask[0] +=1
    #adding 1 at start : this is target (1 is for happy emoji, 0 or -1 for sad face)
    training_set_pos = np.zeros(((np.shape(pos_train)[0],np.shape(embeddings)[1]+1))) + pos_mask
    training_set_neg = np.zeros(((np.shape(neg_train)[0],np.shape(embeddings)[1]+1)))
    vocab = open('data/vocab_cut.txt')
    #for each word, search if it is in pos_train or neg_train
    prevWord =""
    for word_ in vocab:
        word = word_.split("\n")[0]
        #if charac is ecaped, a special regex is used this is why there is a boolean
        if(re.escape(word) != word):
            word= re.escape(word)
        current_emb = embeddings[i]
        for j in range(0,np.shape(pos_train)[0]):
            #if yes, add its embeddings.
            #if word in pos_train[j]:
            if(prevWord != word):
                print(word)
                prevWord=word
            if re.search(r"(?:(?:(?:\b)|^)"+word+"(?:(?=\b)|$)|(?:^|(?:\s))"+word+"(?:$|(?=\s)))",pos_train[j]):
                count = re.findall(r"(?:(?:(?:\b)|^)"+word+"(?:(?=\b)|$)|(?:^|(?:\s))"+word+"(?:$|(?=\s)))",pos_train[j])
                if(j < 20):
                    print(word,pos_train[j],len(count))
                training_set_pos[j,1:np.shape(embeddings)[1]+1] += (len(count)*current_emb)
        for j in range(0,np.shape(neg_train)[0]):
            #if word in neg_train[j]:
            if re.search(r"(?:(?:(?:\b)|^)"+word+"(?:(?=\b)|$)|(?:^|(?:\s))"+word+"(?:$|(?=\s)))",neg_train[j]):
                count = re.findall(r"(?:(?:(?:\b)|^)"+word+"(?:(?=\b)|$)|(?:^|(?:\s))"+word+"(?:$|(?=\s)))",neg_train[j])
                training_set_neg[j,1:np.shape(embeddings)[1]+1] += (len(count)*current_emb)
        i+=1
        if(i%5000 ==0):
            print("5000 done")
    #then divide by number of words (averaging word vector over all words of the tweet)
    for i in range(0,np.shape(embeddings)[1]):
        training_set_pos[:,i+1] = training_set_pos[:,i+1]/word_nbr_per_tweet_pos
        training_set_neg[:,i+1] = training_set_neg[:,i+1]/word_nbr_per_tweet_neg


counting ended
\<user\>
\<user\> <user> i dunno justin read my mention or not . only justin and god knows about that , but i hope you will follow me #believe 15
 1
\<user\> " <user> just put casper in a box ! " looved the battle ! #crakkbitch
 1
\<user\> <user> <user> thanks sir > > don't trip lil mama ... just keep doin ya thang !
 2
\<user\> <user> yay ! ! #lifecompleted . tweet / facebook me to let me know please
 1
\<user\> <user> #1dnextalbumtitle : feel for you / rollercoaster of life . song cocept : life , #yolo , becoming famous ? <3 14 #followmeplz ! <3 x15
 1
\<user\> workin hard or hardly workin rt <user> at hardee's with my future coworker <user>
 2
\<user\> <user> i saw . i'll be replying in a bit .
 1
\<user\> <user> anddd to cheer #nationals2013 ?
 1
\<user\> <user> agreed ! 12 more days left tho
 1
\<user\> like dammm <user> lexis u got a lot to say when ur on twitter lol
 1
\<user\> <user> at home affairs shall do it later
 1
\<user\> <user> a lot of parts of asia . es

KeyboardInterrupt: 

In [ ]:
vocab = ""
for word_ in vocab:
        word = word_.split("\n")[0]
        #if charac is ecaped, a special regex is used this is why there is a boolean
        escaped = False
        listesc = ".^$*+?{}[]\|()" #list of all escaped charactes
        if(word in listesc):
            word= re.escape(word)
            escaped = True
        current_emb = embeddings[i]
        for j in range(0,np.shape(pos_train)[0]):
            #if yes, add its embeddings.
            #if word in pos_train[j]:
            if(prevWord != word):
                print(word)
                prevWord=word
            if(escaped):
                boole = re.search(r""+word,pos_train[j]) #regex if escaped character
            else:
                boole = re.search(r"\b"+word+"\\b",pos_train[j])
                if boole:
                    if(j < 20):
                        print(word,pos_train[j])